# Cleaning Data Anarchy


In [37]:
import glob
import json
import pandas as pd


In [2]:
dfs = []
for filename in glob.glob("../data/Anarchy_*.json"):
    dfs.append(pd.read_json(filename))


In [3]:
df = pd.concat(dfs, ignore_index=True, sort=False)


In [4]:
df.provenance.unique()


array(['PLOS', 'IEEE', 'Springer', 'arXiv', 'Nature'], dtype=object)

In [5]:
len(df.title.unique()), len(df.unique_key.unique())


(791, 825)

In [41]:
entry = pd.DataFrame(entries)


In [67]:
df = pd.concat([df, entry], ignore_index=True, sort=False)


In [74]:
provenance_size = (
    df.groupby(["unique_key", "provenance"])
    .size()
    .reset_index()
    .groupby("provenance")
    .size()
)
provenance_size


provenance
IEEE        229
Manual        1
Nature      184
PLOS         13
Springer    284
arXiv        66
dtype: int64

In [75]:
df.to_json("../data/anarchy_November_2018.json")


In [76]:
df = df[~(df["date"] < 1999)]
df = df[~(df["date"] > 2018)]


In [77]:
df.author = df.author.str.lower()


Duplicate articles
------------------

In [78]:
table = (
    df.groupby(["title", "unique_key"]).size().reset_index().groupby("title").count()
)
duplicates = table[table["unique_key"] > 1]


In [79]:
duplicates_title = df[df["title"].isin(duplicates.index)]["title"].unique()


In [80]:
duplicates_in_arxiv = df[
    (df["title"].isin(duplicates.index)) & (df["provenance"] == "arXiv")
]["title"].unique()


In [81]:
diff = list(set(duplicates_title) - set(duplicates_in_arxiv))


In [82]:
df_without_arxiv = df[~(df["provenance"] == "arXiv")]


In [83]:
df_without_arxiv = df_without_arxiv.drop_duplicates(subset="title")


In [84]:
# df_without_arxiv.to_json('../data/pd_November_2018_without_arxiv.json')


**Drop duplicates.**

In [85]:
articles_to_drop = df[
    (df["title"].isin(duplicates.index)) & (df["provenance"] == "arXiv")
]["unique_key"].unique()


In [86]:
df = df[~df["unique_key"].isin(articles_to_drop)]


In [87]:
# df = df.drop_duplicates(subset='title')


In [88]:
len(df["title"].unique()), len(df["unique_key"].unique())


(747, 777)

**Export clean json.**

In [89]:
df.to_json("../data/anarchy_November_2018_clean.json")
